In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv("./data/train_dataset.csv")

data['is_test'] = False
evaluation = pd.read_csv("./data/evaluation_public.csv")
evaluation['is_test'] = True
sample = pd.read_csv("./data/sample_submission.csv")

all_data = pd.concat([data, evaluation]).reset_index(drop=True)

# 特征处理

In [3]:
all_data['date'] = pd.to_datetime(all_data['time'])
all_data['hour'] = all_data['date'].dt.hour
all_data['year'] = all_data['date'].dt.year
all_data['month'] = all_data['date'].dt.month
all_data['minute'] = all_data['date'].dt.minute
all_data['weekday'] = all_data['date'].dt.weekday
all_data['day'] = all_data['date'].dt.day
all_data['hour'] = all_data['date'].dt.hour
all_data['ts'] = all_data['hour']*3600 + all_data['minute']*60 + all_data['date'].dt.second

In [4]:
import gc
features = [ 'JS_NH3', 'CS_NH3', 'JS_TN', 'CS_TN', 'JS_LL', 'CS_LL', 'MCCS_NH4', 'MCCS_NO3', 'JS_COD', 'CS_COD', 'JS_SW', 'CS_SW', 'B_HYC_NH4', 'B_HYC_XD', 'B_HYC_MLSS', 'B_HYC_JS_DO', 'B_HYC_DO', 'B_CS_MQ_SSLL', 'B_QY_ORP', 'N_HYC_NH4', 'N_HYC_XD', 'N_HYC_MLSS', 'N_HYC_JS_DO', 'N_HYC_DO', 'N_CS_MQ_SSLL', 'N_QY_ORP','weekday','hour', 'ts']
features = [f for f in features if f not in ['time', 'Label1', 'Label2','CS_LL','CS_NH3', 'JS_SW']] #




labels = ['Label1', 'Label2']
train = all_data[~all_data['is_test']].copy(deep=True)
test = all_data[all_data['is_test']].copy(deep=True)
test['is_train'] = False
train['is_train'] = True
data = pd.concat([train, test]).reset_index(drop=True)
del all_data
gc.collect()

for f in features:
    data[f] = data[f].fillna(method='ffill')

# 提取特征的diff值

In [5]:
# tmp_features = list(features)
# add_dict = {}
# for f in tmp_features:
#     data[f'{f}_diff']  = data[f].shift()
#     features.append(f'{f}_diff')


In [6]:
# fff = ['MCCS_NO3', 'B_HYC_NH4','N_HYC_NH4']
# for f in features:
#     data[f] = data[f].fillna(method='ffill')
# for i in range(len(fff)-1):
#     for j in range(i+1, len(fff)):
#         f1 = features[i]
#         f2 = features[j]
#         # feature_jian = f"{f1}_{f2}_jian"
#         feature_radio = f"{f1}_{f2}_radio"
#         data[feature_radio] = data[f1]/data[f2]
#         # data[feature_jian] = data[f1]-data[f2]
#         features.append(feature_radio)
#         # features.append(feature_jian)

In [7]:
df_list = []
i = 0
add_featuers = []
length = 0
for f in ['JS_NH3', 'CS_TN', 'JS_LL', 'JS_COD', 'CS_COD', 'B_HYC_NH4', 'B_HYC_XD', 'B_HYC_MLSS', 'B_HYC_JS_DO', 'B_HYC_DO', 'B_CS_MQ_SSLL',
          'N_HYC_NH4', 'N_HYC_XD', 'N_HYC_MLSS', 'N_HYC_DO', 'N_CS_MQ_SSLL', 'N_QY_ORP']:
    for r in [15]:
        train_rolling = data[f].rolling(window=r, center=False)
        f_mean_name = 'rolling{}_{}_mean'.format(r,f)
        f_max_name = 'rolling{}_{}_max'.format(r,f)
        f_min_name = 'rolling{}_{}_min'.format(r,f)
        f_std_name = 'rolling{}_{}_std'.format(r,f)
        f_corr_name = 'rolling{}_{}_corr'.format(r,f)
        data[f_mean_name] = train_rolling.mean().fillna(0).values
        data[f_max_name] = train_rolling.max().fillna(0).values
        data[f_min_name] = train_rolling.min().fillna(0).values
        data[f_std_name] = train_rolling.std().fillna(0).values
        data[f_corr_name] =  train_rolling.corr().fillna(0).values
        data[f'{f}_{f_mean_name}_cha'] = data[f] - data[f_mean_name]
        data[f'{f}_{f_max_name}_cha'] = data[f] - data[f_max_name]
        data[f'{f}_{f_min_name}_cha'] = data[f] - data[f_min_name]

        if i == 0:
            add_featuers.append(f_mean_name)
            add_featuers.append(f_max_name)
            add_featuers.append(f_min_name)
            add_featuers.append(f_std_name)
            add_featuers.append(f_corr_name)
            add_featuers.append(f'{f}_{f_mean_name}_cha')
            add_featuers.append(f'{f}_{f_max_name}_cha')
            add_featuers.append(f'{f}_{f_min_name}_cha')

    # for r in [5]:
    #     train_rolling = data[f].shift(15).rolling(window=r, center=False)
    #     f_mean_name = 'shift_rolling{}_{}_mean'.format(r,f)
    #     f_max_name = 'shift_rolling{}_{}_max'.format(r,f)
    #     f_min_name = 'shift_rolling{}_{}_min'.format(r,f)
    #     f_std_name = 'shift_rolling{}_{}_std'.format(r,f)
    #     data[f_mean_name] = train_rolling.mean().fillna(0).values
    #     data[f_max_name] = train_rolling.max().fillna(0).values
    #     data[f_min_name] = train_rolling.min().fillna(0).values
    #     data[f_std_name] = train_rolling.std().fillna(0).values
    #     data[f'shift_{f}_{f_mean_name}_cha'] = data[f] - data[f_mean_name]
    #     data[f'shift_{f}_{f_max_name}_cha'] = data[f] - data[f_max_name]
    #     data[f'shift_{f}_{f_min_name}_cha'] = data[f] - data[f_min_name]
    #
    #     if i == 0:
    #         add_featuers.append(f_mean_name)
    #         add_featuers.append(f_max_name)
    #         add_featuers.append(f_min_name)
    #         add_featuers.append(f_std_name)
    #         add_featuers.append(f'shift_{f}_{f_mean_name}_cha')
    #         add_featuers.append(f'shift_{f}_{f_max_name}_cha')
    #         add_featuers.append(f'shift_{f}_{f_min_name}_cha')
features.extend(add_featuers)

# 对所有的特征进行划分
for f in features:
    if f not in ['weekday','hour', 'ts']:
        q = len(data[f].drop_duplicates())
        data[f] = pd.qcut(data[f], q=int(q/10), labels=False, duplicates="drop")

train = data[data['is_train']].reset_index(drop=True)
test = data[~data['is_train']].reset_index(drop=True)

# 删除分布不均衡的特征

In [8]:
train = train.dropna(subset=['Label1', 'Label2']).reset_index(drop=True)
def transform(x: pd.Series, c=20):
    return np.log1p(x/c)

def inverse_transform(x: pd.Series, c = 20):
    return np.expm1(x)*c

train = train.dropna(subset=['Label1', 'Label2']).reset_index(drop=True)
train['Label1'] = transform(train['Label1'])
label_c = 8
train['Label2'] = transform(train['Label2'], c=label_c)

In [9]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
score_list = []
for label in labels:
    train[label] = train[label].fillna(0)
    tmp = train[~train[label].isna()].reset_index(drop=True).copy(deep=True)
    print(len(tmp))
    print(np.max(tmp[label]), np.min(tmp[label]))
    test_size = 4000
    X_train = tmp[features][:-test_size]
    X_test = tmp[features][-test_size:]
    y_train = tmp[label][:-test_size]
    y_test = tmp[label][-test_size:]
    if label == "Label1":
        model = xgb.XGBRegressor(
            max_bin=200,
            boosting="gbdt",
            max_depth=4,
            learning_rate=0.01,
            n_estimators=10000,
            min_child_weight=2,
            subsample = 0.8,
            feature_fraction=0.3,
            bagging_seed=1,
            reg_alpha=1,
            reg_lambda=1,  # 此处不改了
            min_sum_hessian_in_leaf=0.01,
            gpu_id=0,
            tree_method='gpu_hist',
            random_state=1850
        )
        model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], eval_metric=['mae'],
              early_stopping_rounds=150, verbose=200)

        test_pred = model.predict(X_test)
        score_list.append(np.sqrt(mean_squared_error(inverse_transform(y_test), inverse_transform(test_pred))))
        print(f"score_list = {score_list}")
        model2 = xgb.XGBRegressor(
            max_bin=200,
            boosting="gbdt",
            max_depth=4,
            learning_rate=0.01,
            n_estimators=int(1.2*model.best_iteration),
            min_child_weight=2,
            subsample = 0.8,
            feature_fraction=0.3,
            bagging_seed=1,
            reg_alpha=1,
            reg_lambda=1,  # 此处不改了
            min_sum_hessian_in_leaf=0.01,
            gpu_id=0,
            tree_method='gpu_hist',
            random_state=1850
        )
        model2.fit(tmp[features], tmp[label])
        test[label] = inverse_transform(model2.predict(test[features]))
    else:
        model = xgb.XGBRegressor(
            max_depth=5,
            learning_rate=0.1,
            n_estimators=10000,
            min_child_weight=1,
            subsample = 0.4,
            feature_fraction=0.3,
            bagging_seed=1,
            reg_alpha=0.1,
            reg_lambda=0.1,  # 此处不改了
            min_sum_hessian_in_leaf=0.01,
            gpu_id=0,
            tree_method='gpu_hist',
            random_state=1850
        )

        model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], eval_metric=['mae'],
              early_stopping_rounds=150, verbose=200)
        test_pred = model.predict(X_test)
        score_list.append(np.sqrt(mean_squared_error(inverse_transform(y_test, c=label_c), inverse_transform(test_pred, c=label_c))))
        print(f"score_list = {score_list}")
        model2 = xgb.XGBRegressor(
            max_depth=5,
            learning_rate=0.1,
            n_estimators=int(1.2*model.best_iteration),
            min_child_weight=1,
            subsample = 0.4,
            feature_fraction=0.3,
            bagging_seed=0.1,
            reg_alpha=0.1,
            reg_lambda=0.1,  # 此处不改了
            min_sum_hessian_in_leaf=0.01,
            gpu_id=0,
            tree_method='gpu_hist',
            random_state=1850
        )
        model2.fit(tmp[features], tmp[label])

        test[label] = inverse_transform(model2.predict(test[features]), c=label_c)

35068
8.729723218012834 4.510461075505404
[09:34:50] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "bagging_seed", "feature_fraction", "min_sum_hessian_in_leaf" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-mae:6.09050	validation_1-mae:6.03036
[200]	validation_0-mae:0.04053	validation_1-mae:0.07703
[298]	validation_0-mae:0.03594	validation_1-mae:0.07985
score_list = [1100.2393784152046]
[09:34:54] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "bagging_seed", "feature_fraction", "min_sum_hessian_in_leaf" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but


In [10]:
loss = np.mean(score_list)
score = 1000/(1+loss)
print(score)

0.8244496831855618


In [11]:
test[['time'] + labels].to_csv(f"./res/xgboost_res.csv", index=False)